In [16]:
import matplotlib.pyplot as plt
import os as os
import sys as sys
import glob as glob
import re as re
import numpy as np
import shutil as shutil

import paprika
print(paprika.__version__)

from paprika.analysis import fe_calc
from paprika.restraints import DAT_restraint

import pytraj as pt
import parmed as pmd

2018-04-23_16:10:29_-0700-e34840d-0.0.3


In [17]:
import logging
from importlib import reload
reload(logging)

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('Started logging...')

2018-04-25 10:49:17 AM Started logging...


In [18]:
dummy_anchors = [':3', ':4', ':5']
host_anchors  = [':1@C26', ':1@C18', ':1@C8']
guest_anchors = [':2@C6', ':2@C1']

In [19]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 ' \
                '18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 ' \
              '4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 ' \
              '8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 ' \
              '12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 ' \
              '16.80 17.20 17.60 18.00'

windows = [len(attach_fractions), len(pull_string.split()), 0]

In [20]:
g6_complexes = glob.glob('./OA-G6*')

In [21]:
paths = glob.glob(os.path.normpath(os.path.join('./OA-G6-0', 'AMBER', 'APR', 'windows', '*')))

In [22]:
window_list = [os.path.basename(i) for i in paths]
phase_dict = {'a': 'attach', 'p': 'pull', 'r': 'release'}

In [23]:
complx = g6_complexes[0]

In [24]:
traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                os.path.join(complx, 'AMBER', 'solvate.prmtop'))

pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]

print(f'Set pull offset ({pull_initial:1.2f} A)')

pull_distances = [float(i) + pull_initial for i in pull_string.split()]

guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                     [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                     [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                    ]

guest_restraint_targets = [pull_initial,
                           180.0,
                           180.0
                          ]
guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2

hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
               os.path.join(complx, 'AMBER', 'solvate.rst7'),
               structure=True)

guest_restraints = []
for index, atoms in enumerate(guest_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_final'] = guest_restraint_angle_fc
    else:
        this.attach['fc_final'] = guest_restraint_distance_fc
    this.attach['target'] = guest_restraint_targets[index]
    this.attach['fraction_list'] = attach_fractions

    this.pull['target_final'] = guest_restraint_target_final[index]
    this.pull['num_windows'] = windows[1]
    this.initialize()

    guest_restraints.append(this)

structure = pt.load(os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.rst7'), 
                   os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.prmtop'))

stripped = structure.strip(':WAT,:Na+,:Cl-')

Set pull offset (9.98 A)


2018-04-25 10:49:19 AM Calculating attach targets and force constants...
2018-04-25 10:49:19 AM Attach, Method #3
2018-04-25 10:49:19 AM Calculating pull targets and force constants...
2018-04-25 10:49:19 AM Pull, Method #1
2018-04-25 10:49:19 AM Calculating release targets and force constants...
2018-04-25 10:49:19 AM No restraint info set for the release phase! Skipping...
2018-04-25 10:49:19 AM Number of attach windows = 15
2018-04-25 10:49:19 AM Number of pull windows = 46
2018-04-25 10:49:19 AM This restraint will be skipped in the release phase
2018-04-25 10:49:19 AM Assigning atom indices...
2018-04-25 10:49:19 AM There are 1 atoms in the mask :3  ...
2018-04-25 10:49:19 AM There are 1 atoms in the mask :2@C6  ...
2018-04-25 10:49:19 AM Calculating attach targets and force constants...
2018-04-25 10:49:19 AM Attach, Method #3
2018-04-25 10:49:19 AM Calculating pull targets and force constants...
2018-04-25 10:49:19 AM Pull, Method #1
2018-04-25 10:49:19 AM Calculating release ta

In [26]:
fecalc = fe_calc()
fecalc.prmtop = stripped.topology
fecalc.trajectory = 'prod.*.nc'
fecalc.path = os.path.join(complx, 'AMBER', 'APR','windows')
fecalc.restraint_list = guest_restraints
fecalc.collect_data()
fecalc.compute_free_energy()


2018-04-25 10:49:48 AM Replacing ./OA-G6-0/AMBER/APR/windows/a014 with ./OA-G6-0/AMBER/APR/windows/p000 in ['./OA-G6-0/AMBER/APR/windows/a000', './OA-G6-0/AMBER/APR/windows/a001', './OA-G6-0/AMBER/APR/windows/a002', './OA-G6-0/AMBER/APR/windows/a003', './OA-G6-0/AMBER/APR/windows/a004', './OA-G6-0/AMBER/APR/windows/a005', './OA-G6-0/AMBER/APR/windows/a006', './OA-G6-0/AMBER/APR/windows/a007', './OA-G6-0/AMBER/APR/windows/a008', './OA-G6-0/AMBER/APR/windows/a009', './OA-G6-0/AMBER/APR/windows/a010', './OA-G6-0/AMBER/APR/windows/a011', './OA-G6-0/AMBER/APR/windows/a012', './OA-G6-0/AMBER/APR/windows/a013', './OA-G6-0/AMBER/APR/windows/a014'] for `continuous_apr`...
2018-04-25 10:49:48 AM Reading restraint data for ./OA-G6-0/AMBER/APR/windows/a000...
2018-04-25 10:49:48 AM Reading restraint data for ./OA-G6-0/AMBER/APR/windows/a000...
2018-04-25 10:49:48 AM Reading restraint data for ./OA-G6-0/AMBER/APR/windows/a000...
2018-04-25 10:49:48 AM Reading restraint data for ./OA-G6-0/AMBER/APR/

2018-04-25 10:49:50 AM Reading restraint data for ./OA-G6-0/AMBER/APR/windows/p043...
2018-04-25 10:49:50 AM Reading restraint data for ./OA-G6-0/AMBER/APR/windows/p044...
2018-04-25 10:49:50 AM Reading restraint data for ./OA-G6-0/AMBER/APR/windows/p045...
2018-04-25 10:49:52 AM attach: computing convergence for mbar-blocking
2018-04-25 10:50:18 AM pull: computing convergence for mbar-blocking
2018-04-25 10:50:18 AM Skipping free energy calculation for release


In [28]:
fecalc.results

{'attach': {'mbar-block': {'fe': 3.8189555600631233,
   'sem': 0.04222363133604452,
   'fe_matrix': array([[ 0.        ,  0.11280369,  0.21410346, ...,  2.95209542,
            3.44783889,  3.81895556],
          [-0.11280369,  0.        ,  0.10129977, ...,  2.83929173,
            3.3350352 ,  3.70615187],
          [-0.21410346, -0.10129977,  0.        , ...,  2.73799195,
            3.23373543,  3.6048521 ],
          ...,
          [-2.95209542, -2.83929173, -2.73799195, ...,  0.        ,
            0.49574348,  0.86686014],
          [-3.44783889, -3.3350352 , -3.23373543, ..., -0.49574348,
            0.        ,  0.37111667],
          [-3.81895556, -3.70615187, -3.6048521 , ..., -0.86686014,
           -0.37111667,  0.        ]]),
   'sem_matrix': array([[0.        , 0.0039158 , 0.00728001, ..., 0.03820034, 0.04044858,
           0.04222363],
          [0.0039158 , 0.        , 0.00337463, ..., 0.03588269, 0.03826771,
           0.04013949],
          [0.00728001, 0.00337463, 0

In [31]:
traj = pt.load(os.path.join(complx, 'AMBER', 'APR','windows', 'a000', 'prod*'), stripped.topology)

In [32]:
traj

pytraj.Trajectory, 2000 frames: 
Size: 0.009209 (GB)
<Topology: 206 atoms, 5 residues, 5 mols, PBC with box type = ortho>
           

In [36]:
traj[0:int(0.1 * traj.n_frames)]

pytraj.Trajectory, 200 frames: 
Size: 0.000921 (GB)
<Topology: 206 atoms, 5 residues, 5 mols, PBC with box type = ortho>
           

In [37]:
from paprika.analysis import read_restraint_data